In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/books']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv', error_bad_lines=False)
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [1]:
import os
import numpy as np
# import pandas as pd
exec(os.environ['IREWR_IMPORTS'])

# ALEX: remove path printing
# print(os.listdir("./input"))

In [2]:
csv_file = "./input/books.scaled.csv"
df = pd.read_csv(csv_file, error_bad_lines=False)
df.head()

/tmp/ipykernel_130432/1532740798.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(csv_file, error_bad_lines=False)
Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13



,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [3]:
df.shape

(11123, 12)

In [4]:
df.dtypes

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                  int64
language_code          object
  num_pages             int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
dtype: object

In [5]:
rating_df = df.groupby(['authors'])['average_rating'].mean().reset_index().round(3)
rating_df.head()

,authors,average_rating
0,A.B. Yehoshua/Hillel Halkin,3.60
1,A.D.P. Briggs/Leo Tolstoy/Fyodor Dostoyevsky,3.76
2,A.E. Cunningham/Harlan Ellison/Charles F. Mill...,4.15
3,A.J. Jacobs,3.76
4,A.M. Homes,3.28


In [6]:
rating_df.sort_values(['average_rating', 'authors'], ascending=[False, True]).reset_index(drop=True).head(10)

,authors,average_rating
0,Aristophanes/F.W. Hall/W.M. Geldart,5.0
1,Chris Green/Chris Wright/Paul Douglas Gardner,5.0
2,Dennis Adler/R.L. Wilson,5.0
3,Elena N. Mahlow,5.0
4,Ian Martin/Katie Elliott,5.0
5,James E. Campbell,5.0
6,John Diamond,5.0
7,Julie Sylvester/David Sylvester,5.0
8,Keith Donohue,5.0
9,Laura Driscoll/Alisa Klayman-Grodsky/Eric ...,5.0


In [7]:
# ALEX: make notebook run
# pd.cut(df['# num_pages'], [-1, 100, 200, 300]).head()
pd.cut(df['  num_pages'], [-1, 100, 200, 300]).head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name:   num_pages, dtype: category
Categories (3, interval[int64, right]): [(-1, 100] < (100, 200] < (200, 300]]

In [8]:
# ALEX: make notebook run
# range_df = df.groupby(pd.cut(df['# num_pages'], [-1, 100, 200, 300]))
range_df = df.groupby(pd.cut(df['  num_pages'], [-1, 100, 200, 300]))
range_df = range_df[['ratings_count']]
range_df.sum().reset_index()

,num_pages,ratings_count
0,"(-1, 100]",4777739
1,"(100, 200]",24629394
2,"(200, 300]",43922582


In [9]:
# ALEX: make notebook run
# df.groupby(['authors'])['# num_pages'].max().reset_index()
df.groupby(['authors'])['  num_pages'].max().reset_index()

,authors,num_pages
0,A.B. Yehoshua/Hillel Halkin,576
1,A.D.P. Briggs/Leo Tolstoy/Fyodor Dostoyevsky,256
2,A.E. Cunningham/Harlan Ellison/Charles F. Mill...,232
3,A.J. Jacobs,389
4,A.M. Homes,256
...,...,...
6634,Émile Zola/Henri Mitterand,462
6635,Émile Zola/Robert Lethbridge/Elinor Dorday,592
6636,Émile Zola/Robin Buss/Brian Nelson,438
6637,Émile Zola/Roger Pearson,592


In [10]:
# ALEX: make notebook run
# avg_df = df.groupby(['authors'])['# num_pages'].mean().reset_index()
# avg_df['# num_pages'] = avg_df['# num_pages'].astype('int64')
avg_df = df.groupby(['authors'])['  num_pages'].mean().reset_index()
avg_df['  num_pages'] = avg_df['  num_pages'].astype('int64')
avg_df

,authors,num_pages
0,A.B. Yehoshua/Hillel Halkin,416
1,A.D.P. Briggs/Leo Tolstoy/Fyodor Dostoyevsky,256
2,A.E. Cunningham/Harlan Ellison/Charles F. Mill...,232
3,A.J. Jacobs,389
4,A.M. Homes,256
...,...,...
6634,Émile Zola/Henri Mitterand,462
6635,Émile Zola/Robert Lethbridge/Elinor Dorday,592
6636,Émile Zola/Robin Buss/Brian Nelson,438
6637,Émile Zola/Roger Pearson,528
